In [3]:
import os
import pandas as pd
from openai import OpenAI

In [4]:
#OPEN AI KEY

os.environ["OPENAI_API_KEY"] = "sk-proj-rBVJlXCeswQsTIRXgosYT3BlbkFJRupEcTiKXToudT9qUFwU"

OPENAI_API_KEY: str = os.environ.get("OPENAI_API_KEY")


In [4]:
occupations_list = [
    'ICT%20sales%20professional',
    'Marketing%20manager',
    'Product%20analyst',
    'Product%20manager',
    'Product%20designer',
    'Business%20Intelligence%20professional',
    'Infraestructure%20engineer',
    'Infraestructure%20engineer',
    'Infrastructure%20engineer',
    'Computer%20Systems%20Analyst',
    'Software%20infrastructure%20architect',
    'Web%20developer',
    'Software%20developer',
    'App%20developer',
    'User%20interface%20designer',
    'Software%20engineer',
    'Software%20architect',
    'Software%20Quality%20Assurance%20Analysts%20and%20Testers',
    'Embedded%20systems%20engineer',
    'Web%20and%20Digital%20Interface%20Designers',
    'Database%20infrastructure%20engineer',
    'Network%20architect',
    'Database%20Administrators',
    'Database%20Architects',
    'Network%20and%20Computer%20Systems%20Administrator',
    'Artificial%20intelligence%20engineer',
    'Machine%20Learning%20engineer',
    'Data%20science%20engineer',
    'Data%20analyst',
    'Data%20scientist',
    'Artificial%20Intelligence%20scientist',
    'Data%20architect',
    'ICT%20security%20specialist',
    'IT%20security%20operations',
    'Information%20Security%20Analysts',
    'Product%20Security%20and%20IT%20Security%20Integration%20Specialist',
    'Product%20risk%20specialist',
    'Security%20architect',
    'Database%20support%20engineer',
    'Data%20center%20operations%20engineer',
    'Support%20systems%20engineer',
    'Computer%20Network%20Support%20Specialists'
]


In [23]:
# Read dict KSA's

dict_file = "Golden/dict_ksao_v12-junio-2024.csv"
golden = pd.read_csv(dict_file)
listText = list(golden["Text"])
setText  = set(listText)

relations_df = pd.DataFrame()
relations_df['source'] = None
relations_df['target'] = None
relations_df['ksao'] = None
relations_df['value'] = None
relations_df['origin'] = None

relations_set = set()
occ_count = 1

for occ in occupations_list:
    file_path = f'../GoogleForJobs/infocomm/7-junio-sfco-processed/{occ}_competences_llm.csv'
    if os.path.isfile(file_path) and os.path.getsize(file_path) > 1:
        print(f"{occ_count}.- Analyzing occupation: {occ}")
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        for x in df["competences_llm"]:
            for item in x.split("\n"):
                # TODO: Checar los casos en que en las competencias no están en lista numérica con punto '.', por ej:
                # 
                # 1. Troubleshooting skills
                # 2. Computer security knowledge
                # 3. High-level experience in:
                #     a. Very high performance/traffic web architectures
                #     b. Configuration management
                #     c. Large scale deployment methods
                #     d. Ansible
                # 
                # 
                #   o también:
                # 
                # - Administration
                # - Maintenance
                # - Technical support
                # - Local area network (LAN)
                # - Connectivity
                # - Departmental host system
                # - Network operating s
                # 
                # 
                # 
                #   o también:
                # 
                # 
                # 4. Cyber Security Software tool proficiency
                #     a. Darktrace
                #     b. EDR
                #     c. Security Operations Center (SOC)
                #     d. SEIM
                # 
                # 
                # 
                # 
                # 
                # Checar: if ". " in item or "- " in item:
                if ". " in item:
                    ksa = item.split(". ")[1]
                    found = False
                    
                    #Busca en "Standard text"
                    # for golden_ksa in golden["Standard text"]:
                    #     if golden_ksa == ksa:
                    #         index = golden.index[golden["Standard text"] == golden_ksa].tolist()[0]
                    #         _ksa = golden.at[index, 'Label']
                    #         record = {'source': occ, 'target': ksa, 'ksao': _ksa,'value': 1, 'origin': 'Standard text'}
                    #         relations_set.add(tuple(record.items()))
                    #         relations_df = pd.concat([relations_df, pd.DataFrame([record])], ignore_index=True)
                    #         found = True
                    #         break
                    
                    # if not found:
                    #     #Busca en "Text"
                    #     for golden_ksa in golden["Text"]:
                    #         if golden_ksa == ksa:
                    #             index = golden.index[golden["Text"] == golden_ksa].tolist()[0]
                    #             _text = golden.at[index, 'Text']
                    #             _label = golden.at[index, 'Label']
                    #             record = {'occ': occ, 'text': _text, 'label': _label, 'value': 1}
                    #             relations_set.add(tuple(record.items()))
                    #             relations_df = pd.concat([relations_df, pd.DataFrame([record])], ignore_index=True)
                    #             found = True
                    #             break


                    #Busca en "Text"
                    if(ksa in setText):
                        index = listText.index(ksa)
                        _ksa = ksa
                        _ksa = golden.at[index, 'Text']
                        _ksao = golden.at[index, 'Label']
                        #record = {'source': occ, 'target': _ksa, 'ksao': _ksao, 'value': 1, 'origin': 'Text'}
                        record = {'occ': occ, 'text': _ksa, 'label': _ksao, 'value': 1}
                        relations_set.add(tuple(record.items()))
                        relations_df = pd.concat([relations_df, pd.DataFrame([record])], ignore_index=True)
                        found = True
                    
                    if not found:
                        #Usar nuestro fine-tuned model para obtener clasificación de KSAO y agregar.
                        # Initialize OpenAI client
                        client = OpenAI()

                        # Call OpenAI's completion API with the current text
                        completion = client.chat.completions.create(
                            # Usar el modelo finetuned el 4 de Junio de 2024 id: 9WV7nHBF
                            model="ft:gpt-3.5-turbo-0125:personal::9WV7nHBF",
                            messages=[
                                {"role": "system", "content": "Your job is to identify if the text belongs to knowledge, skill, ability or other based on the KSA taxonomy. Ability is different from skill as it is innate and difficult to measure. You cannot teach ability whereas you can teach skill."},
                                {"role": "user", "content": ksa}
                            ]
                        )
                        # Get the predicted output
                        predicted_output = completion.choices[0].message.content
                        
                        parts = predicted_output.split(':')
                        # If there are two parts and the second part is not empty
                        if len(parts) == 2 and parts[1].strip():
                            # Capitalize the first letter of the second part and strip any leading/trailing whitespace
                            capitalized = parts[1].strip().capitalize()
                        else:
                            capitalized = predicted_output

                        if capitalized in ['Knowledge','Skill','Ability','Other']:
                            record = {'occ': occ, 'text': ksa, 'label': capitalized, 'value': 1}
                            relations_set.add(tuple(record.items()))
                            relations_df = pd.concat([relations_df, pd.DataFrame([record])], ignore_index=True)

                            #Add record to golden KSA to prevent further API calls.
                            new_row = pd.DataFrame({'Label': [capitalized], 'Text': [ksa]})
                            golden_df2 = pd.concat([golden, new_row], ignore_index=True)
                            golden_df2.to_csv(dict_file, index=False)
                            golden = pd.read_csv(dict_file)

    else:
        print(f"{occ_count}.- {occ} - 0")

    occ_count += 1
    

1.- ICT%20sales%20professional - 0
2.- Analyzing occupation: Marketing%20manager
3.- Analyzing occupation: Product%20analyst
4.- Analyzing occupation: Product%20manager
5.- Analyzing occupation: Product%20designer
6.- Analyzing occupation: Business%20Intelligence%20professional
7.- Analyzing occupation: Infraestructure%20engineer
8.- Infrastructure%20engineer - 0
9.- Analyzing occupation: Computer%20Systems%20Analyst
10.- Analyzing occupation: Software%20infrastructure%20architect
11.- Analyzing occupation: Web%20developer
12.- Analyzing occupation: Software%20developer
13.- Analyzing occupation: App%20developer
14.- Analyzing occupation: User%20interface%20designer
15.- Analyzing occupation: Software%20engineer


In [6]:
import pandas as pd
import os
import re

# Define la función para limpiar y extraer competencias
def extract_competences(x):
    competences = []
    for item in x.split("\n"):
        if ". " in item:
            competence = item.split(". ")[1].strip()
            if competence:
                competences.append(competence)
        elif "- " in item:
            competence = item.split("- ")[1].strip()
            if competence:
                competences.append(competence)
    return competences

# Carga el archivo golden KSA
golden = pd.read_csv("Golden/dict_ksao_v12-junio-2024.csv")
setText = set(golden["Text"].unique())

relations_df = pd.DataFrame(columns=['source', 'target', 'ksao', 'value', 'origin'])

file_path = 'indeed_competences/competences_indeed_June_SanFrancisco.csv'
if os.path.isfile(file_path) and os.path.getsize(file_path) > 1:
    df = pd.read_csv(file_path)
    for index, row in df.iterrows():
        competences = extract_competences(row['competences_llm'])
        found = False
        if ksa in setText:
            ksa_index = listText.index(ksa)
            _ksao = golden.at[ksa_index, 'Label']
            record = {'source': row['Occ'], 'target': ksa, 'ksao': _ksao, 'value': 1, 'origin': 'Text'}
            relations_set.add(tuple(record.items()))
            relations_df = pd.concat([relations_df, pd.DataFrame([record])], ignore_index=True)
            found = True
        
        if not found:
            # Usar el modelo finetuned para clasificación
            try:
                response = client.completions.create(
                    model="ft:gpt-3.5-turbo-0125:personal::9WV7nHBF",
                    messages=[
                        {"role": "system", "content": "Your job is to identify if the text belongs to knowledge, skill, ability or other based on the KSA taxonomy. Ability is different from skill as it is innate and difficult to measure. You cannot teach ability whereas you can teach skill."},
                        {"role": "user", "content": ksa}
                    ]
                )
                classification = response.choices[0].text.strip()
                
                # Capitalize and check validity
                if classification.capitalize() in ['Knowledge', 'Skill', 'Ability', 'Other']:
                    record = {'source': row['JobTitle'], 'target': ksa, 'ksao': classification.capitalize(), 'value': 1, 'origin': 'Model'}
                    relations_set.add(tuple(record.items()))
                    relations_df = pd.concat([relations_df, pd.DataFrame([record])], ignore_index=True)
                    
                    # Update Golden file
                    new_row = pd.DataFrame({'Label': [classification.capitalize()], 'Text': [ksa]})
                    golden = pd.concat([golden, new_row], ignore_index=True)
                    golden.to_csv(dict_file, index=False)
            except Exception as e:
                print(f"Failed to classify {ksa} due to: {e}")
else:
    print("File not found or is empty")

# Guardar los resultados en un CSV
relations_df.to_csv('output_relations.csv', index=False)


In [ ]:
grouped_df = relations_df.groupby(['occ', 'text', 'label', 'value']).size().reset_index(name='count')
grouped_df = grouped_df.drop(columns=['value'])
grouped_df

In [21]:
grouped_df.to_csv('tax-infocomm-jsearch-1w-junio-SFCO-2024.csv', index=False)
